Based on the tutorial from https://muon-tutorials.readthedocs.io/en/latest/single-cell-rna-atac/brain3k/1-Processing-and-Integration.html

In [1]:
import mudatasets as mds

In [2]:
import scipy.io
import pandas as pd
import numpy as np
from anndata import AnnData
from mudata import MuData
import scanpy as sc
import muon as mu
from muon import atac as ac
from os.path import join
from vitessce.data_utils import (
    VAR_CHUNK_SIZE,
    optimize_adata,
    adata_to_multivec_zarr,
)

## Load the data

In [ ]:
mdata = mds.load("brain3k_multiome", full=True)
mdata.var_names_make_unique()
mdata

## 1. RNA

## QC

In [ ]:
# `rna` will point to `mdata['rna']`
# unless we copy it
rna = mdata['rna']

In [ ]:
rna.var['mt'] = rna.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(rna, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(rna, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

In [ ]:
mu.pp.filter_obs(rna, 'n_genes_by_counts', lambda x: (x >= 200) & (x < 8000))
mu.pp.filter_obs(rna, 'total_counts', lambda x: x < 40000)
mu.pp.filter_obs(rna, 'pct_counts_mt', lambda x: x < 2)

In [ ]:
sc.pl.violin(rna, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

## Scaling and normalization

In [ ]:
rna.layers["counts"] = rna.X.copy()
sc.pp.normalize_total(rna, target_sum=1e4)
sc.pp.log1p(rna)
# rna.raw = rna
rna.layers["lognorm"] = rna.X.copy()

## Identify highly-variable genes

In [ ]:
sc.pp.highly_variable_genes(rna, min_mean=0.02, max_mean=4, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(rna)

In [ ]:
sc.pp.scale(rna, max_value=10)

In [ ]:
sc.tl.pca(rna, svd_solver='arpack')
sc.pl.pca(rna, color=['NRCAM', 'SLC1A2', 'SRGN', 'VCAN'])

In [ ]:
sc.pl.pca_variance_ratio(rna, log=True)

In [ ]:
sc.pp.neighbors(rna, n_neighbors=10, n_pcs=20)
sc.tl.leiden(rna, resolution=.5)

In [ ]:
sc.tl.umap(rna, spread=1., min_dist=.5, random_state=11)
sc.pl.umap(rna, color="leiden", legend_loc="on data")

## Cell type annotation

In [ ]:
sc.tl.rank_genes_groups(rna, 'leiden', method='t-test')

In [ ]:
result = rna.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.set_option('display.max_columns', 50)
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(10)

In [ ]:
sc.pl.rank_genes_groups(rna, n_genes=20, sharey=False)

In [ ]:
sc.pl.umap(rna, color=["PLP1", "CNP", "CTNNA3"])

In [ ]:
sc.pl.umap(rna, color=["SLC1A2", "SRGN", "VCAN"], title=["SLC1A2 (astrocytes)", "SRGN (microglia)", "VCAN (OPCs)"])

In [ ]:
new_cluster_names = {
    "0": "oligodendrocyte",
    "1": "oligodendrocyte",
    "3": "oligodendrocyte",
    "5": "oligodendrocyte",
    "14": "oligodendrocyte",
    "4": "OPC",
    "8": "microglia",
    "2": "astrocyte",
    "10": "astrocyte",
    "11": "astrocyte",
    "12": "astrocyte",
    "6": "excitatory_LAMP5",
    "13": "excitatory_RORB",
    "7": "inhibitory_LHX6",
    "9": "inhibitory_ADARB2",
    "15": "inhibitory_ADARB2",
}

In [ ]:
rna.obs['celltype'] = [new_cluster_names[cl] for cl in rna.obs.leiden.astype("str").values]
rna.obs.celltype = rna.obs.celltype.astype("category")

In [ ]:
sc.pl.umap(rna, color="celltype")

## 2. ATAC

In [ ]:
atac = mdata.mod['atac']

In [ ]:
sc.pp.calculate_qc_metrics(atac, percent_top=None, log1p=False, inplace=True)

In [ ]:
mu.pl.histogram(atac, ['n_genes_by_counts', 'total_counts'], linewidth=0)

In [ ]:
mu.pp.filter_var(atac, 'n_cells_by_counts', lambda x: x >= 10)

In [ ]:
mu.pp.filter_obs(atac, 'total_counts', lambda x: (x >= 1000) & (x <= 80000))
mu.pp.filter_obs(atac, 'n_genes_by_counts', lambda x: (x >= 100) & (x <= 30000))

In [ ]:
mu.pl.histogram(atac, ['n_genes_by_counts', 'total_counts'], linewidth=0)

In [ ]:
ac.pl.fragment_histogram(atac, region='chr1:1-2000000')

In [ ]:
ac.tl.nucleosome_signal(atac, n=1e6)

In [ ]:
mu.pl.histogram(atac, "nucleosome_signal", linewidth=0)

In [ ]:
# Check TSS enrichment
ac.tl.get_gene_annotation_from_rna(mdata['rna']).head(3)  # accepts MuData with 'rna' modality or mdata['rna'] AnnData directly

In [ ]:
tss = ac.tl.tss_enrichment(mdata, n_tss=1000)  # by default, features=ac.tl.get_gene_annotation_from_rna(mdata)

In [ ]:
ac.pl.tss_enrichment(tss)

In [ ]:
atac.layers["counts"] = atac.X.copy()
sc.pp.normalize_total(atac, target_sum=1e4)
sc.pp.log1p(atac)
atac.layers["lognorm"] = atac.X.copy()

In [ ]:
sc.pp.highly_variable_genes(atac, min_mean=0.05, max_mean=1.5, min_disp=.5)
sc.pl.highly_variable_genes(atac)

In [ ]:
np.sum(atac.var.highly_variable)

In [ ]:
sc.pp.scale(atac, max_value=10)
sc.tl.pca(atac, svd_solver='arpack')
ac.pl.pca(atac, color=['NRCAM', 'SLC1A2', 'SRGN', 'VCAN'], layer='lognorm', func='mean')

In [ ]:
sc.pl.pca_variance_ratio(atac, log=True)

In [ ]:
sc.pp.neighbors(atac, n_neighbors=10, n_pcs=20)
sc.tl.leiden(atac, resolution=.5)

In [ ]:
sc.tl.umap(atac, spread=1., min_dist=.5, random_state=11)
sc.pl.umap(atac, color="leiden", legend_loc="on data")

## Marker genes and cell types

In [ ]:
ac.tl.rank_peaks_groups(atac, 'leiden', method='t-test')

In [ ]:
result = atac.uns['rank_genes_groups']
groups = result['names'].dtype.names

try:
    pd.set_option("max_columns", 50)
except:
    # https://pandas.pydata.org/pandas-docs/stable/user_guide/options.html
    pd.set_option("display.max_columns", 50)

pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'genes', 'pvals']}).head(10)

In [ ]:
mu.pp.filter_obs(atac, "leiden", lambda x: ~x.isin(["9"]))

In [ ]:
new_cluster_names = {
    "0": "oligodendrocyte",
    "1": "oligodendrocyte",
    "3": "OPC",
    "7": "microglia",
    "2": "astrocyte",
    "8": "astrocyte",
    "4": "excitatory",
    "5": "inhibitory1",
    "6": "inhibitory2",
    "10": "unk"
}

In [ ]:
atac.obs['celltype'] = [new_cluster_names[cl] for cl in atac.obs.leiden.astype("str").values]
atac.obs.celltype = atac.obs.celltype.astype("category")

In [ ]:
sc.pl.umap(atac, color="celltype")

## 3. Multi-omics integration
Discard cells that are not in both modalities.

In [ ]:
mdata.update()

In [ ]:
mu.pp.intersect_obs(mdata)

In [ ]:
mu.tl.mofa(mdata, n_factors=20, outfile="brain3k_mofa_model.hdf5", gpu_mode=True)

In [ ]:
sc.pp.neighbors(mdata, use_rep="X_mofa")
sc.tl.umap(mdata, random_state=1)

In [ ]:
mdata.obsm["X_mofa_umap"] = mdata.obsm["X_umap"]

In [ ]:
mu.pl.embedding(mdata, basis="X_mofa_umap", color=["rna:celltype", "atac:celltype"])

In [ ]:
# Reference: https://github.com/scverse/muon/issues/65
mdata.mod["atac"].uns = {}
mdata.mod["rna"].uns = {}
mdata.uns = {}

In [ ]:
rna.var.gene_ids = rna.var.gene_ids.astype("str")
atac.var.gene_ids = atac.var.gene_ids.astype("str")

In [ ]:
atac.obsm["X_hvg"] = atac[:, atac.var["highly_variable"]].copy().X
rna.obsm["X_hvg"] = rna[:, rna.var["highly_variable"]].copy().X

In [ ]:
mdata.mod["atac"] = optimize_adata(
    atac,
    obs_cols=["leiden", "celltype"],
    obsm_keys=["X_pca", "X_umap", "X_hvg"],
    var_cols=["gene_ids", "feature_types", "genome", "interval", "highly_variable"],
    layer_keys=["counts", "lognorm"]
)
mdata.mod["rna"] = optimize_adata(
    rna,
    obs_cols=["leiden", "celltype"],
    obsm_keys=["X_pca", "X_umap", "X_hvg"],
    var_cols=["gene_ids", "feature_types", "genome", "interval", "highly_variable"],
    layer_keys=["counts", "lognorm"]
)

In [ ]:
# Fix issue during writing to zarr - this column contains bool and NaN
mdata.var["rna:mt"] = mdata.var["rna:mt"].astype(str)

In [ ]:
# TODO: sort var axis by genome (ATAC) and hierarchical clustering (RNA)

In [ ]:
mdata

In [ ]:
mdata.write_zarr(join("data", "brain3k_processed.mdata.zarr"))

## Configure visualization

In [1]:
!pwd

/Users/mkeller/research/dbmi/vitessce/paper-figures/multiome/src


In [7]:
multivec_zarr = join("..", "data", "brain3k.multivec.zarr")
adata_to_multivec_zarr(mdata.mod["atac"], multivec_zarr, obs_set_col="celltype", obs_set_name="Cell Type", layer_key="lognorm")

In [8]:
from vitessce import (
    VitessceConfig,
    ViewType as vt,
    CoordinationType as ct,
    FileType as ft,
    AnnDataWrapper,
    MultivecZarrWrapper,
)

In [9]:
vc = VitessceConfig(schema_version="1.0.15", name='Multiome data', description='RNA+ATAC')

In [11]:
rna_zarr = join("..", "data", "brain3k_processed.mdata.zarr", "mod", "rna")
atac_zarr = join("..", "data", "brain3k_processed.mdata.zarr", "mod", "atac")
joint_zarr = join("..", "data", "brain3k_processed.mdata.zarr")

In [12]:
dataset = vc.add_dataset(name='RNA+ATAC').add_object(AnnDataWrapper(
    # We run add_object with adata_path=rna_zarr first to add the cell-by-gene matrix and associated metadata.
    adata_path=rna_zarr,
    obs_embedding_paths=["obsm/X_umap"],
    obs_embedding_names=["UMAP"],
    obs_set_paths=["obs/celltype"],
    obs_set_names=["Cell Type"],
    obs_feature_matrix_path="obsm/X_hvg",
    feature_filter_path="var/highly_variable",
    # To be explicit that the features represent genes and gene expression, we specify that here.
    coordination_values={
        "featureType": "gene",
        "featureValueType": "expression"
    }
)).add_object(AnnDataWrapper(
    # We next run add_object with adata_path=adt_zarr to add the cell-by-ADT matrix and associated metadata.
    adata_path=atac_zarr,
    obs_embedding_paths=["obsm/X_umap"],
    obs_embedding_names=["UMAP"],
    obs_set_paths=["obs/celltype"],
    obs_set_names=["Cell Type"],
    obs_feature_matrix_path="obsm/X_hvg",
    feature_filter_path="var/highly_variable",
    # If the features do not represent genes and gene expression, we specify alternate values here.
    coordination_values={
        "featureType": "peak",
        "featureValueType": "count"
    }
)).add_object(MultivecZarrWrapper(
    # We next run add_object with adata_path=adt_zarr to add the cell-by-ADT matrix and associated metadata.
    zarr_path=multivec_zarr,
))

In [13]:
genomic_profiles = vc.add_view(vt.GENOMIC_PROFILES, dataset=dataset)
scatter = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping = "UMAP")
cell_sets = vc.add_view(vt.OBS_SETS, dataset=dataset)

vc.layout(genomic_profiles / (scatter | cell_sets));


In [15]:
vw = vc.widget(height=800)
vw

VitessceWidget(config={'version': '1.0.15', 'name': 'Multiome data', 'description': 'RNA+ATAC', 'datasets': [{…

In [ ]:
umap_scatterplot_by_rna = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")
umap_scatterplot_by_atac = vc.add_view(vt.SCATTERPLOT, dataset=dataset, mapping="UMAP")

gene_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)
peak_list = vc.add_view(vt.FEATURE_LIST, dataset=dataset)

rna_heatmap = vc.add_view(vt.HEATMAP, dataset=dataset).set_props(transpose=False)
atac_heatmap = vc.add_view(vt.HEATMAP, dataset=dataset).set_props(transpose=False)

In [ ]:
# We need to specify which of the two features (i.e., genes or tags) the different plots correspond to.
# We also need to make sure the selection of genes and tags are scoped to only the corresponding plots,
# and we want to make sure the color mappings are independent for each modality.
coordination_types = [ct.FEATURE_TYPE, ct.FEATURE_VALUE_TYPE, ct.FEATURE_SELECTION, ct.OBS_COLOR_ENCODING, ct.FEATURE_VALUE_COLORMAP_RANGE]
vc.link_views([umap_scatterplot_by_rna, gene_list, rna_heatmap], coordination_types, ["gene", "expression", None, 'cellSetSelection', [0.0, 0.3]])
vc.link_views([umap_scatterplot_by_atac, peak_list, atac_heatmap], coordination_types, ["peak", "count", None, 'cellSetSelection', [0.0, 1.0]])

# We can link the two scatterplots on their zoom level and (X,Y) center point so that zooming/panning is coordinated.
vc.link_views([umap_scatterplot_by_rna, umap_scatterplot_by_atac], [ct.EMBEDDING_ZOOM, ct.EMBEDDING_TARGET_X, ct.EMBEDDING_TARGET_Y], [3, 0, 0])

In [ ]:
# We define a layout for the plots using two rows.
# In the first row, we add the three gene-related visualizations,
# and in the second row, we add the three ADT-related visualizations.
vc.layout(
    (rna_heatmap | (umap_scatterplot_by_rna | gene_list))
    / (atac_heatmap | (umap_scatterplot_by_atac | peak_list))
);

In [ ]:
vw = vc.widget()
vw